<h2>Looking for Recipes ...</h2>
<p>www.simplyrecipes.com</p>
<p>www.allrecipes.com</p>

In [1]:
import requests
from bs4 import BeautifulSoup as bs

In [2]:
recipe_list = []

In [3]:
def simplyrecipes_search(vegetable, recipe_list):
    # Search for recipes that feature a user-specified vegetable on the
    # simplyrecipes website. Append each recipe to recipe_list.
    # Return a response status code [200 => all went well].
    
    url = "https://www.simplyrecipes.com/?s=" + vegetable
    response = requests.get(url)
    print("Scraping :", url, " - ", response.status_code)
    
    if response.status_code == 200:
        
        soup = bs(response.text, 'html.parser')
        
        for axx in soup.find_all('li'):
    
            if axx.has_attr('itemprop'):
                if axx['itemprop'] == "itemListElement":
            
                    for axxp in axx.find_all('a'):
                
                        # Link to the recipe.
                        link = axxp['href']
                
                        # Title.
                        title = axxp['title']
                
                    # Summary description.   
                    for axxp in axx.find_all('div', class_="summary"):
                        summary = axxp.text.lstrip()
            
                    # A picture of the dish.
                    for axxp in axx.find_all('div', class_="featured-image"):
                        image = axxp.img['src']
        
                    recipe_list.append({"link":link, "title":title, "image":image, "summary":summary, "featured vegetable":vegetable})
        
    return response.status_code

In [4]:
def allrecipes_search(vegetable, recipe_list):
    # Search for recipes that feature a user-specified vegetable on the
    # allrecipes website. Append each recipe to recipe_list.
    # Return a response status code [200 => all went well]
    
    url = "https://www.allrecipes.com/search/results/?wt=" + vegetable + "&sort=re"
    response = requests.get(url)
    print("Scraping :", url, " - ", response.status_code)
    
    if response.status_code == 200:

        soup = bs(response.text, 'html.parser')

        for axx in soup.find_all('article', class_ = "fixed-recipe-card"):
            # Link to the recipe.
            axxp = axx.find('a')
            link = axxp['href']

            # Title.
            axxp = axx.find('span', class_="fixed-recipe-card__title-link")
            title = axxp.text

            # A picture of the dish.
            axxp = axx.find('img', class_="fixed-recipe-card__img")
            image = axxp['data-original-src']

            # Summary description.
            axxp = axx.find('div', class_="fixed-recipe-card__description")
            summary = axxp.text

            recipe_list.append({"link":link, "title":title, "image":image, "summary":summary, "featured vegetable":vegetable})
        
    return response.status_code

In [8]:
# Search for recipes and add them to the recipe_list.
recipe_list = []
allrecipes_search("spinach", recipe_list)
simplyrecipes_search("spinach", recipe_list)
allrecipes_search("broccoli", recipe_list)
simplyrecipes_search("broccoli", recipe_list)
allrecipes_search("squash", recipe_list)
simplyrecipes_search("squash", recipe_list)
allrecipes_search("asparagus", recipe_list)
simplyrecipes_search("asparagus", recipe_list)
simplyrecipes_search("lentils", recipe_list)
simplyrecipes_search("pepper", recipe_list)

Scraping : https://www.allrecipes.com/search/results/?wt=spinach&sort=re  -  200
Scraping : https://www.simplyrecipes.com/?s=spinach  -  403
Scraping : https://www.allrecipes.com/search/results/?wt=broccoli&sort=re  -  200
Scraping : https://www.simplyrecipes.com/?s=broccoli  -  403
Scraping : https://www.allrecipes.com/search/results/?wt=squash&sort=re  -  200
Scraping : https://www.simplyrecipes.com/?s=squash  -  403
Scraping : https://www.allrecipes.com/search/results/?wt=asparagus&sort=re  -  200
Scraping : https://www.simplyrecipes.com/?s=asparagus  -  403
Scraping : https://www.simplyrecipes.com/?s=lentils  -  403
Scraping : https://www.simplyrecipes.com/?s=pepper  -  200


200

<h2>Mongo DB Creation for Recipes</h2>

In [31]:
# Import the pymongo library which will allow us to create and maintain a Mongo DB.
import pymongo

# Create connection variable
conn = 'mongodb://localhost:27017'

# Pass connection to the pymongo instance.
client = pymongo.MongoClient(conn)

# Connect to a database. Create a database called mars.
db = client.ETL

# Drops collection 'recipes' if available to remove duplicates.
db.recipes.drop()

db.recipes.insert_many(recipe_list)